In [2]:
!pip install pandas
!pip install seaborn

# boas práticas: criar uma célula com todas as bibliotecas necessárias
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

    100% |████████████████████████████████| 26.3MB 25kB/s  eta 0:00:01
    100% |████████████████████████████████| 512kB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 12.2MB 55kB/s  eta 0:00:01
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 184kB 2.3MB/s ta 0:00:01
    100% |████████████████████████████████| 31.2MB 20kB/s  eta 0:00:01
    100% |████████████████████████████████| 12.6MB 51kB/s  eta 0:00:01
    100% |████████████████████████████████| 61kB 5.5MB/s eta 0:00:01
    100% |████████████████████████████████| 952kB 700kB/s eta 0:00:01
  Running setup.py bdist_wheel for seaborn ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/26/0a/44/53ddd89769e62f7c6691976375b86c6492e7dd20a2d3970e32
Successfully built seaborn
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading 

In [3]:
# Turmas 2017.2
# http://dados.ufrn.br/dataset/turmas

# turmas
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'

# matrículas em componentes em 2017.2
notas_url = 'http://bit.do/notas_ufrn_2017_2'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# turmas 2017.2
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.2
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

In [4]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

# Pegando apenas as disciplinas do IMD
disciplinas = disciplinas[disciplinas.unidade_responsavel == 'INSTITUTO METROPOLE DIGITAL']

In [5]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario", "ano", "periodo"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
disciplinas_turma = pd.merge(turmas, disciplinas, on='id_componente')[colunas]
# Caso a disciplina tenha mais de um docente ela aparece mais de uma vez, esse trecho faz com que cada turma apareça
# apenas uma vez.
#disciplinas_turma = disciplinas_turma[~disciplinas_turma.id_turma.duplicated()]

In [21]:
colunas_relevantes = ["id_turma", "codigo", "nome", "descricao_horario", "ano", 
                      "periodo", "media_final", "numero_total_faltas", "descricao", "discente"]
# Merge de turma com as notas
notas_turma = pd.merge(disciplinas_turma, notas, on='id_turma')[colunas_relevantes]

# Como são 3 unidades, cada aluno aparece 3 vezes, porém estamos interessados apenas no resultado final
# Obs: Por algum motivo essa linha está removendo registros não duplicados
#notas_turma = notas_turma[~notas_turma.discente.duplicated()]

In [19]:
notas_fmc = notas_turma[notas_turma.nome == 'FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO II']
notas_lp1 = notas_turma[notas_turma.nome == 'LINGUAGEM DE PROGRAMAÇÃO I']
notas_lp2 = notas_turma[notas_turma.nome == 'LINGUAGEM DE PROGRAMAÇÃO II']
notas_ple1 = notas_turma[notas_turma.nome == 'PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I']

resultado_fmc = notas_fmc.groupby("descricao").aggregate({"numero_total_faltas": ["mean", "min", "max"]})
resultado_lp1 = notas_lp1.groupby("descricao").aggregate({"numero_total_faltas": ["mean", "min", "max"]})
resultado_lp2 = notas_lp2.groupby("descricao").aggregate({"numero_total_faltas": ["mean", "min", "max"]})
resultado_ple1 = notas_ple1.groupby("descricao").aggregate({"numero_total_faltas": ["mean", "min", "max"]})

resultado_lp1

numero_total_faltas            
                                                mean   min   max
descricao                                                       
APROVADO                                    6.933333   0.0  18.0
APROVADO POR NOTA                          10.516129   0.0  20.0
CANCELADO                                  18.028571   0.0  62.0
DESISTENCIA                                      NaN   NaN   NaN
INDEFERIDO                                       NaN   NaN   NaN
REPROVADO                                  15.600000   8.0  20.0
REPROVADO POR MÉDIA E POR FALTAS           40.088235  22.0  68.0
REPROVADO POR NOTA                         10.666667  10.0  12.0
TRANCADO                                    2.000000   0.0   4.0